In [1]:
%matplotlib tk

import argparse
import gym
import datetime
import os
import random
import tempfile
import numpy as np
import pickle

import ray
from ray import tune
from ray.tune.logger import Logger, UnifiedLogger, pretty_print
from ray.rllib.env.multi_agent_env import make_multi_agent
from ray.rllib.examples.models.shared_weights_model import TF2SharedWeightsModel
from ray.rllib.models import ModelCatalog
from ray.rllib.utils.framework import try_import_tf
from ray.rllib.utils.test_utils import check_learning_achieved
from ray.rllib.agents.ppo import ppo
from ray.rllib.models import ModelCatalog
from environment_rllib_3d import MyEnv
from settings.initial_settings import *
from settings.reset_conditions import reset_conditions
#from modules.models import MyConv2DModel_v0B_Small_CBAM_1DConv_Share
from modules.models import DenseNetModelLarge
from tensorflow.keras.utils import plot_model
from modules.savers import save_conditions
from utility.result_env import render_env

import matplotlib.pyplot as plt
import matplotlib
import tensorflow as tf
import cv2
import ctypes
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 
warnings.filterwarnings('ignore', category=matplotlib.MatplotlibDeprecationWarning)

PROJECT = "UCAV"
TRIAL_ID = 2
TRIAL = 'test_' + str(TRIAL_ID)
EVAL_FREQ = 10
NUM_EVAL = 10
CONTINUAL = True

def custom_log_creator(custom_path, custom_str):
    timestr = datetime.datetime.today().strftime("%Y-%m-%d_%H-%M-%S")
    logdir_prefix = "{}_{}".format(custom_str, timestr)

    def logger_creator(config):
        if not os.path.exists(custom_path):
            os.makedirs(custom_path)
        logdir = tempfile.mkdtemp(prefix=logdir_prefix, dir=custom_path)
        return UnifiedLogger(config, logdir, loggers=None)

    return logger_creator

ray.shutdown()
ray.init(ignore_reinit_error=True, log_to_driver=False)

ModelCatalog.register_custom_model('my_model', DenseNetModelLarge)

# config = {"env": MyEnv,
#           "num_workers": NUM_WORKERS,
#           "num_gpus": NUM_GPUS,
#           "num_cpus_per_worker": NUM_CPUS_PER_WORKER,
#           "num_sgd_iter": NUM_SGD_ITER,
#           "lr": LEARNING_RATE,
#           "gamma": GAMMA,  # default=0.99
#           "model": {"custom_model": "my_model"}
#           # "framework": framework
#           }  # use tensorflow 2
config = {"env": MyEnv,"num_gpus": 0,"num_workers": 0, "num_cpus_per_worker": 0,"num_gpus": 0}

# PPOTrainer()は、try_import_tfを使うと、なぜかTensorflowのeager modeのエラーになる。

trainer = ppo.PPOTrainer(config=config,
                         logger_creator=custom_log_creator(
                             os.path.expanduser("./" + PROJECT + "/logs"), TRIAL))

if CONTINUAL:
    # Continual learning: Need to specify the checkpoint
    model_path = PROJECT + '/checkpoints/' + TRIAL + '/checkpoint_000166/checkpoint-166'
    trainer.restore(checkpoint_path=model_path)

# Instanciate the evaluation env
eval_env = MyEnv({})
record_mode = 0

2021-11-25 08:45:34,702	INFO services.py:1265 -- View the Ray dashboard at http://127.0.0.1:8265
2021-11-25 08:45:48,406	INFO trainer.py:714 -- Tip: set framework=tfe or the --eager flag to enable TensorFlow eager execution
2021-11-25 08:45:48,406	INFO ppo.py:159 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
2021-11-25 08:45:48,421	INFO trainer.py:728 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
2021-11-25 08:45:52,061	WARNING util.py:55 -- Install gputil for GPU system monitoring.
2021-11-25 08:45:52,778	INFO trainable.py:383 -- Restored on 128.52.204.204 from checkpoint: UCAV/checkpoints/test_2/checkpoint_000166/checkpoint-166
2021-11-25 08:45:52,778	INFO trainable.py:390 -- Current state after restoring: {'_iteration': 166, '_timesteps_total': None, '_time_total': 123307.41022372246, '_episodes_total'

In [ ]:
for i in range(NUM_EVAL):
    # print(f'\nEvaluation {i}:')
    obs = eval_env.reset()
    done = False

    step_num = 0
    fig = plt.figure(1,figsize=(8.0, 6.0))
    ESC = 0x1B          # ESCキーの仮想キーコード
    trajectory_length = 100
    env_blue_pos = [0]
    env_red_pos = [0]
    env_mrm_pos = [0]
    if record_mode == 0:
        file_name = "test_num" +str(i)
        video = cv2.VideoWriter(file_name+'.mp4',0x00000020,20.0,(800,600))

    while True:
        action_dict = {}
        for j in range(eval_env.blue_num):
            if not eval_env.blue[j].hitpoint == 0:
                action_dict['blue_' + str(j)] = trainer.compute_action(obs['blue_' + str(j)])

        obs, rewards, dones, infos = eval_env.step(action_dict)
        env_blue_pos_temp, env_red_pos_temp, env_mrm_pos_temp= render_env.copy_from_env(eval_env)
        env_blue_pos.append(env_blue_pos_temp)
        env_red_pos.append(env_red_pos_temp)
        env_mrm_pos.append(env_mrm_pos_temp)
        if step_num == 0:
            del env_blue_pos[0]
            del env_red_pos[0]
            del env_mrm_pos[0]

        hist_blue_pos = np.vstack(env_blue_pos)
        hist_red_pos = np.vstack(env_red_pos)
        hist_mrm_pos = np.vstack(env_mrm_pos)
        plt.clf()
        render_env.rend_3d(eval_env,hist_blue_pos,"b",1)
        render_env.rend_3d(eval_env,hist_red_pos,"r",1)
        render_env.rend_3d(eval_env,hist_mrm_pos,"k",1)
        plt.pause(.05)
        step_num = step_num + 1

        done = dones["__all__"]
        if record_mode == 0:
            img = np.array(fig.canvas.renderer.buffer_rgba())
            img = cv2.cvtColor(img, cv2.COLOR_RGBA2BGR)
            # cv2.imshow('test', img)
            # cv2.waitKey(1)
            # cv2.destroyAllWindows()
            video.write(img.astype('uint8'))
        elif record_mode == 1:
            env.render()
        elif record_mode == 2:
            True #将来のため処理予約

        #env_blue_pos_temp, env_red_pos_temp, env_mrm_pos_temp = render_env.copy_from_env(eval_env)

        #env_blue_pos.append(env_blue_pos_temp)
        #env_red_pos.append(env_red_pos_temp)
        #env_mrm_pos.append(env_mrm_pos_temp)
        #step_num = step_num + 1
        # エピソードの終了処理
        if dones['__all__']:
            # print(f'all done at {env.steps}')
            if record_mode == 0:
                video.release()
            break

    #del env_blue_pos[0]
    #del env_red_pos[0]
    #del env_mrm_pos[0]

    #hist_blue_pos = np.vstack(env_blue_pos)
    #hist_red_pos = np.vstack(env_red_pos)
    #hist_mrm_pos = np.vstack(env_mrm_pos)

    #f = open(results_file,'wb')
    #pickle.dump(emv_blue_pos,f)
    #pickle.dump(emv_red_pos,f)
    #pickle.dump(emv_mrm_pos,f)
    #f.close()

    if record_mode == 0:
        video.release()

ray.shutdown()

2021-11-25 08:45:57,644	WARNING deprecation.py:39 -- DeprecationWarning: `compute_action` has been deprecated. Use `compute_single_action` instead. This will raise an error in the future!


blue_1DOWN
here
False True 394 0.001 0.5980000000000004
False True 394 -2.0 -1.5549999999999997
blue_1DOWN
here
False True 258 0.001 0.45800000000000035
False True 258 -2.0 -1.5329999999999997
blue_1DOWN
here
False True 308 0.001 0.45800000000000035
False True 308 -1.0 -0.5749999999999997
blue_0DOWN
here
False True 122 -1.0 -0.8749999999999999
False True 122 0.003 0.1900000000000001
blue_0DOWN
here
False True 74 -1.0 -0.8989999999999999
False True 74 0.001 0.07400000000000002
blue_0DOWN
here
False True 259 -1.0 -0.6659999999999997
False True 259 0.003 0.4010000000000003
blue_0DOWN
here
False True 121 -1.0 -0.7979999999999998
False True 121 0.001 0.09300000000000007
blue_1DOWN
here
False True 99 0.001 0.05900000000000006
False True 99 -2.0 -1.92
